In [102]:
import numpy as np
import pandas as pd
import os
from PIL import Image
from tqdm import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import timm

In [103]:
class BlindnessDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, test=False):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.test = test
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0] + '.png')
        image = Image.open(img_name)
        
        if self.transform:
            image = self.transform(image)
        
        if self.test:
            return image
        else:
            label = int(self.annotations.iloc[idx, 1])
            return image, label

In [104]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [105]:
test_csv_file = '/kaggle/input/aptos2019-blindness-detection/test.csv'
test_root_dir = '/kaggle/input/aptos2019-blindness-detection/test_images'
test_dataset = BlindnessDataset(test_csv_file, test_root_dir, transform=transform, test=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [106]:

model_paths = {
    'resnet18': "/kaggle/input/aptos_ensamble-models/pytorch/ensamble_v1/1/resnet18(WD_1e-3)_aptos.pth",
    #'efficientnet_b5': "/kaggle/input/aptos_ensamble-models/pytorch/ensamble_v1/1/efficientnet_b5.pth",
    #'inception_resnet_v2': "/kaggle/input/aptos_ensamble-models/pytorch/ensamble_v1/1/inception_resnet_v2.pth",
    #'inception_v4': "/kaggle/input/aptos_ensamble-models/pytorch/ensamble_v1/1/inception_v4.pth",
    'seresnext50_32x4d': "/kaggle/input/aptos_ensamble-models/pytorch/ensamble_v1/1/seresnext50_32x4d.pth",
    'seresnext101_32x4d': "/kaggle/input/aptos_ensamble-models/pytorch/ensamble_v1/1/seresnext101_32x4d.pth"
}
"""
model_paths = {
    'seresnext101_32x4d': "/kaggle/input/aptos_ensamble-models/pytorch/ensamble_v1/1/seresnext101_32x4d.pth"
}
"""

model_names = {
    'resnet18': 'resnet18',
    #'efficientnet_b5': 'efficientnet_b5',
    #'inception_resnet_v2': 'inception_resnet_v2',
    #'inception_v4': 'inception_v4',
    'seresnext50_32x4d': 'seresnext50_32x4d',
    'seresnext101_32x4d': 'seresnext101_32x4d'
}


In [107]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models_list = []

# Initialize and load weights for each model using timm
for model_key, path in model_paths.items():
    model_name = model_names[model_key]
    model = timm.create_model(model_name, pretrained=False, num_classes=5)
    model.load_state_dict(torch.load(path))
    model.to(device)
    model.eval()
    models_list.append(model)

In [108]:
all_outputs = []

# Perform predictions with each model
with torch.no_grad():
    for images in tqdm(test_loader):
        images = images.to(device)
        outputs = [model(images).unsqueeze(0) for model in models_list]
        outputs = torch.cat(outputs)
        averaged_outputs = torch.mean(outputs, dim=0)
        all_outputs.extend(averaged_outputs.cpu().numpy())

# Convert the averaged outputs to predictions
all_outputs = np.array(all_outputs)
final_predictions = np.argmax(all_outputs, axis=1)
print(final_predictions.shape)

100%|██████████| 121/121 [01:17<00:00,  1.57it/s]

(1928,)


In [109]:
submission_df = pd.DataFrame({
    'id_code': pd.read_csv(test_csv_file)['id_code'],
    'diagnosis': final_predictions
})

submission_df.to_csv('submission.csv', index=False)